1. Попробуйте обучить три сети представленные в тетради.
(Разморозкой слоев заниматься не надо, разберем разморозку на уроке.)
Сохраните модели.

2. Напишите функцию принимающую изображение и дающую прогноз (кот /собака) с использованием всех трех моделей.

Сделаем необходимые импорты для работы в Google Colab.

In [1]:
from google.colab import drive
from google.colab import files

Монтируем Google Диск в виртуальную машину Google Colab.

In [2]:
drive.mount('/GD',force_remount=True)

Mounted at /GD


Содержимое папки

In [3]:
!ls

sample_data


Копируем на диск архив с фото и распакуем его.

In [ ]:
!cp -r /GD/'My Drive'/PetImages.zip .
!unzip PetImages.zip

Содержимое папки

In [5]:
!ls

PetImages  PetImages.zip  sample_data


Теперь она содержит запакованный архив и распакованную папку из архива

Внутри папки лежат две папки с train и test сетами.

In [6]:
!ls PetImages

test  train


Создадим папки для validation сета

In [7]:
!mkdir PetImages/validation_data
!mkdir PetImages/validation_data/Dog
!mkdir PetImages/validation_data/Cat

In [8]:
!ls PetImages/validation_data

Cat  Dog


Отделим 10% картинок из train сета для валидации.

In [9]:
import os, shutil
import random as rn

In [10]:
train_dirs = ['PetImages/train/Dog', 
              'PetImages/train/Cat']

validation_dirs = ['PetImages/validation_data/Dog', 
                   'PetImages/validation_data/Cat']

Почистим папки от недопустимых файлов

In [11]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError
import os
i=0
for pa in train_dirs:
    path = Path(pa).rglob("*.jpg")
    for img_p in path:
        try:
            img = PIL.Image.open(img_p)
        except PIL.UnidentifiedImageError:
                i+=1
                print(img_p, i)
                os.remove(img_p)

Из каждой тренировочной папки выберем случайным образом 10% файлов, скопируем их в validation_data и удалим их из исходной папки.

In [ ]:
for i in range(len(train_dirs)):
    fnames = os.listdir(train_dirs[i])
    toCopy = len(fnames) // 10

    files_to_copy = []
    for j in range(toCopy):
        files_to_copy.append(rn.choice(fnames))
         
    for f in files_to_copy:
        src = os.path.join(train_dirs[i],f)
        dst = os.path.join(validation_dirs[i], f)
        shutil.copyfile(src,dst)

    
    for f in files_to_copy:
        try:
            src = os.path.join(train_dirs[i],f)
            os.remove(src)
            print(src + ' deleted')
        except:
            pass


Осмотрим папки.

In [ ]:
!ls PetImages/validation_data/Dog

In [ ]:
!ls PetImages/validation_data/Cat

## подготовка генераторов

Импортируем keras для сборки моделей

In [15]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers

Подготовим генератор изображений, который будет извлекать изображения из размеченных папок, приводить их к нужному размеру и готовить из них батчи.

In [16]:
Pic_Y, Pic_X = 180, 180
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Подготовим генераторы для train и test множеств.


In [17]:
train_datagen = ImageDataGenerator(rescale = 1./255)
                            
train_generator = train_datagen.flow_from_directory('PetImages/train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 18503 images belonging to 2 classes.


In [18]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('PetImages/test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 4542 images belonging to 2 classes.


Соберем модель.

In [19]:
model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", 
                        input_shape = (Pic_Y, Pic_X, 3)))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model.add(layers.Flatten())
model.add(layers.Dense(1, activation="sigmoid"))


In [20]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 41, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 20, 20, 128)      0

In [22]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.h5",
        save_best_only=True,
        monitor="val_accuracy")]

In [23]:
history = model.fit(train_generator, epochs=10, validation_data=test_generator,
    callbacks=callbacks)

Epoch 1/10
290/290 [==============================] - ETA: 0s - loss: 0.6865 - accuracy: 0.5745

/usr/local/lib/python3.9/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


290/290 [==============================] - 102s 307ms/step - loss: 0.6865 - accuracy: 0.5745 - val_loss: 0.5824 - val_accuracy: 0.6900
Epoch 2/10
290/290 [==============================] - 87s 302ms/step - loss: 0.5503 - accuracy: 0.7202 - val_loss: 0.5090 - val_accuracy: 0.7428
Epoch 3/10
290/290 [==============================] - 88s 303ms/step - loss: 0.4533 - accuracy: 0.7888 - val_loss: 0.3726 - val_accuracy: 0.8329
Epoch 4/10
290/290 [==============================] - 87s 299ms/step - loss: 0.3721 - accuracy: 0.8336 - val_loss: 0.3607 - val_accuracy: 0.8344
Epoch 5/10
290/290 [==============================] - 89s 309ms/step - loss: 0.3062 - accuracy: 0.8665 - val_loss: 0.3305 - val_accuracy: 0.8536
Epoch 6/10
290/290 [==============================] - 86s 296ms/step - loss: 0.2552 - accuracy: 0.8919 - val_loss: 0.3024 - val_accuracy: 0.8699
Epoch 7/10
290/290 [==============================] - 85s 294ms/step - loss: 0.2083 - accuracy: 0.9143 - val_loss: 0.2398 - val_accuracy: 0.

Сохраним модель на Google Disk.

In [24]:
!cp convnet_from_scratch.h5 /GD/'My Drive'/convnet_from_scratch.h5
!ls

convnet_from_scratch.h5  PetImages  PetImages.zip  sample_data


## Добавляем расширение данных 

In [25]:
model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", 
                        input_shape = (Pic_Y, Pic_X, 3)))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model.add(layers.Flatten())
model.add(layers.Dense(1, activation="sigmoid"))


In [26]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 89, 89, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 41, 41, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 20, 20, 128)     

In [28]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="augmentation_model.h5",
        save_best_only=True,
        monitor="val_loss")]

Данный генератор будет случайным образом искажать обучающие фотографии

In [29]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 25,
                                   shear_range = 0.15,
                                   zoom_range = 0.3,
                                   fill_mode = 'nearest')
                            
train_generator = train_datagen.flow_from_directory('PetImages/train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 18503 images belonging to 2 classes.


In [30]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('PetImages/test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 4542 images belonging to 2 classes.


In [31]:
history = model.fit(train_generator, epochs=10, validation_data=test_generator,
    callbacks=callbacks)

Epoch 1/10
290/290 [==============================] - 211s 722ms/step - loss: 0.6694 - accuracy: 0.5849 - val_loss: 0.5783 - val_accuracy: 0.6944
Epoch 2/10
290/290 [==============================] - 208s 717ms/step - loss: 0.5865 - accuracy: 0.6903 - val_loss: 0.5053 - val_accuracy: 0.7528
Epoch 3/10
290/290 [==============================] - 208s 719ms/step - loss: 0.5100 - accuracy: 0.7511 - val_loss: 0.4329 - val_accuracy: 0.7983
Epoch 4/10
290/290 [==============================] - 211s 729ms/step - loss: 0.4665 - accuracy: 0.7768 - val_loss: 0.4002 - val_accuracy: 0.8190
Epoch 5/10
290/290 [==============================] - 209s 720ms/step - loss: 0.4218 - accuracy: 0.8030 - val_loss: 0.3759 - val_accuracy: 0.8404
Epoch 6/10
290/290 [==============================] - 208s 719ms/step - loss: 0.3846 - accuracy: 0.8254 - val_loss: 0.3278 - val_accuracy: 0.8622
Epoch 7/10
290/290 [==============================] - 208s 717ms/step - loss: 0.3548 - accuracy: 0.8432 - val_loss: 0.2613 -

Сохраним модель на Google Disk.

In [32]:
!cp augmentation_model.h5 /GD/'My Drive'/augmentation_model.h5
!ls

augmentation_model.h5	 PetImages	sample_data
convnet_from_scratch.h5  PetImages.zip


## Перенос знаний!

Соберем последнюю модель. Она тоже будет обучаться на аугментированных данных

In [33]:
conv_base = tf.keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))


58889256/58889256 [==============================] - 4s 0us/step


In [34]:
conv_base.trainable = False

In [35]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1, activation = 'sigmoid'))

In [36]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 5, 5, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 12800)             0         
                                                                 
 dense_2 (Dense)             (None, 1)                 12801     
                                                                 
Total params: 14,727,489
Trainable params: 12,801
Non-trainable params: 14,714,688
_________________________________________________________________


In [38]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="VGG_FIX_model.h5",
        save_best_only=True,
        monitor="val_loss")]

In [39]:
history = model.fit(train_generator, epochs=10, validation_data=test_generator, 
    callbacks=callbacks)

Epoch 1/10
290/290 [==============================] - 232s 778ms/step - loss: 0.3832 - accuracy: 0.8240 - val_loss: 0.3390 - val_accuracy: 0.8490
Epoch 2/10
290/290 [==============================] - 220s 757ms/step - loss: 0.2833 - accuracy: 0.8775 - val_loss: 0.1868 - val_accuracy: 0.9188
Epoch 3/10
290/290 [==============================] - 223s 768ms/step - loss: 0.2598 - accuracy: 0.8903 - val_loss: 0.1886 - val_accuracy: 0.9177
Epoch 4/10
290/290 [==============================] - 222s 765ms/step - loss: 0.2455 - accuracy: 0.8969 - val_loss: 0.1896 - val_accuracy: 0.9214
Epoch 5/10
290/290 [==============================] - 222s 764ms/step - loss: 0.2364 - accuracy: 0.8998 - val_loss: 0.2050 - val_accuracy: 0.9181
Epoch 6/10
290/290 [==============================] - 221s 761ms/step - loss: 0.2299 - accuracy: 0.9042 - val_loss: 0.2680 - val_accuracy: 0.9011
Epoch 7/10
290/290 [==============================] - 221s 762ms/step - loss: 0.2292 - accuracy: 0.9049 - val_loss: 0.1864 -

Сохраним модель на Google Disk.

In [40]:
!cp VGG_FIX_model.h5 /GD/'My Drive'/VGG_FIX_model.h5
!ls

augmentation_model.h5	 PetImages	sample_data
convnet_from_scratch.h5  PetImages.zip	VGG_FIX_model.h5


Оценим модели на проверочном множестве.

In [41]:
val_datagen = ImageDataGenerator(rescale = 1./255)
                            
val_generator = val_datagen.flow_from_directory('PetImages/validation_data',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 1953 images belonging to 2 classes.


In [42]:
for m in ('convnet_from_scratch.h5',
          'augmentation_model.h5',
          'VGG_FIX_model.h5'):
    model = tf.keras.models.load_model(m)
    test_loss, test_acc = model.evaluate(val_generator)
    print(f"Test accuracy for model {m.split('.')[0]} is : {test_acc:.3f}")

31/31 [==============================] - 8s 241ms/step - loss: 0.2571 - accuracy: 0.9007
Test accuracy for model convnet_from_scratch is : 0.901
31/31 [==============================] - 7s 203ms/step - loss: 0.2576 - accuracy: 0.8920
Test accuracy for model augmentation_model is : 0.892
31/31 [==============================] - 11s 354ms/step - loss: 0.2131 - accuracy: 0.9160
Test accuracy for model VGG_FIX_model is : 0.916
